<img src="images/array.png" width="25%" align="right">

Arrays
======

Dask array provides a parallel, larger-than-memory, numpy ndarray using blocked algorithms.

*  **Parallel**: Uses all of the cores on your computer
*  **Larger-than-memory**:  Lets you work on datasets that are larger than your available memory by breaking up your array into many small pieces, operating on those pieces in an order that minimizes the memory footprint of your computation, and effectively streaming data from disk.

**Documentation**

* http://dask.readthedocs.org/en/latest/array.html
* http://dask.readthedocs.org/en/latest/array-api.html

Blocked Algorithms
---------------------

A *blocked algorithm* executes on a large dataset by breaking it up into many small blocks.

For example, consider taking the sum of a billion numbers

#### Create random dataset

In [ ]:
# create data if it doesn't already exist
from prep import random_array
random_array()  

# Load data with h5py
import h5py
import os
f = h5py.File(os.path.join('data', 'random.hdf5'))
dset = f['/x']

### Compute sum using blocked algorithm

Here we compute the sum of this large array on disk by 

1.  Computing the sum of each 1,000,000 sized chunk of the array
2.  Computing the sum of the 1,000 intermediate sums

In [ ]:
# Compute sum of large array, one million numbers at a time
sums = []
for i in range(0, 1000000000, 1000000):
    chunk = dset[i: i + 1000000]  # pull out numpy array
    sums.append(chunk.sum())

total = sum(sums)
print total

### Exercise:  Compute mean of positive elements using blocked algorithm

Alter the algorithm above to compute number of elements above one by doing the following

1.  Compute the number of elements above one in each 1,000,000 sized chunk of the array
2.  Compute the sum of the 1,000 intermediate counts

In [ ]:
# Count number of elements greater than one

In [ ]:
# Solution
%load solutions/Array-01.py

`dask.array` contains these algorithms
--------------------------------------------

Dask.array does these kinds of tricks to operate on large datasets.  It extends beyond these simple linear examples to full N-Dimensional algorithms and a decent subset of the NumPy interface.

#### Create `dask.array` object

You can create an `dask.array.Array` object with the `da.from_array` function.  This function accepts

1.  Any object that supports NumPy slicing, like `dset[1000: 2000]`
2.  A chunk size to tell us how to block up our array

In [ ]:
import dask.array as da
x = da.from_array(dset, chunks=(1000000,))

#### Manipulate `dask.array` object as you would a numpy array

Now that we have an `Array` we perform standard numpy-style computations like arithmetic, mathematics, slicing, reductions, etc..

In [ ]:
result = x.sum()
result

#### Compute result

Dask.array objects are lazily evaluated.  Operations like `.sum` build up a graph of blocked tasks to execute.  

We ask for the final result with a call to `.compute()`.  This triggers the actual computation.

In [ ]:
result.compute()

### Exercise:  Count of all elements greater than one

Find out how many elements are greater than one using 

    (x > 1).sum()


Performance and Parallelism
-------------------------------

<img src="images/fail-case.gif" width="40%" align="right">

In our first examples we used `for` loops to walk through the array one block at a time.  For simple operations like `sum` this is optimal.  However for complex operations we may want to traverse through the array differently.  In particular we may want the following:

1.  Use multiple cores in parallel
2.  Chain operations on a single blocks before moving on to the next one

Dask.array translates your array operations into a graph of inter-related tasks with data dependencies between them.  Dask then executes this graph in parallel with multiple threads.  We'll discuss more about this in the next section.



### Example

1.  Construct a 20000x20000 array of normally distributed random values broken up into 1000x1000 sized chunks
2.  Take the mean along one axis
3.  Take every 100th element

In [ ]:
import numpy as np
import dask.array as da
x = da.random.normal(10, 0.1, size=(20000, 20000),   # 400 million element array 
                              chunks=(1000, 1000))   # Cut into 1000x1000 sized chunks
y = x.mean(axis=0)[::100]                            # Perform NumPy-style operations

In [ ]:
x.nbytes / 1e9  # Gigabytes of the input processed lazily

In [ ]:
%%time
y.compute()     # Time to compute the result

Performance comparision
---------------------------

The following experiment was performed on a heavy personal laptop.  Your performance may vary.  If you attempt the NumPy version then please ensure that you have more than 4GB of main memory.

#### NumPy: 19s, Needs gigabytes of memory

```python
import numpy as np

%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
y

CPU times: user 19.6 s, sys: 160 ms, total: 19.8 s
Wall time: 19.7 s
```

#### Dask Array: 4s, Needs megabytes of memory

```python
import dask.array as da

%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
y = x.mean(axis=0)[::100] 
y.compute() 

CPU times: user 29.4 s, sys: 1.07 s, total: 30.5 s
Wall time: 4.01 s
```

### Exercise:  Meteorological data

There is 2GB of somewhat artifical weather data in HDF5 files in `data/weather-big/*.hdf5`.  We'll use the `h5py` library to interact with this data and `dask.array` to compute on it.

Our goal is to visualize the average temperature on the surface of the Earth for this month.  This will require a mean over all of this data.  We'll do this in the following steps

1.  Create `h5py.Dataset` objects for each of the days of data on disk
2.  Wrap these with `da.from_array` calls
3.  Stack these datasets with a call to `da.stack`
4.  Compute the mean along the time axis with the `.mean()` method
5.  Visualize the result with `matplotlib.pyplot.imshow`

In [ ]:
import h5py
from glob import glob
import os

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename)['/t2m'] for filename in filenames]
dsets[0]

In [ ]:
dsets[0][:5, :5]  # Slicing into h5py.Dataset object gives a numpy array

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(16, 8))
plt.imshow(dsets[0], cmap='RdBu_r')

#### Integrate with `dask.array`

Make a list of `dask.array` objects out of your list of `h5py.Dataset` objects using the `da.from_array` function with a chunk size of `(1000, 1000)`.

#### Stack this list of `dask.array` objects into a single `dask.array` object with `da.stack`

Stack these along the first axis so that the shape of the resulting array is `(31, 5760, 11520)`.

#### Plot the mean of this array along the time (`0th`) axis

In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.imshow(..., cmap='RdBu_r')

### Solution

In [ ]:
%%load solutions/Array-02.py

### Exercise:  Subsample and store

In the above exercise the result of our computation is small, so we can call `compute` safely.  Sometimes our result is still too large to fit into memory and we want to save it to disk.  In these cases you can use one of the following two functions

1.  `da.store`: Store dask.array into any object that supports numpy setitem syntax, e.g.

        f = h5py.File('myfile.hdf5')
        output = f.create_dataset(shape=..., dtype=...)
        
        da.store(my_dask_array, output)
        
2.  `da.to_hdf5`: A specialized function that creates and stores a `dask.array` object into an `HDF5` file.

        da.to_hdf5('data/myfile.hdf5', '/output', my_dask_array)
        
The task in this exercise is to using numpy step slicing to subsample the full dataset by a factor of two in both the latitude and longitude direction and then store this result to disk using one of the functions listed above.

As a reminder, Python slicing takes three elements

    start:stop:step

    >>> L = [1, 2, 3, 4, 5, 6, 7]
    >>> L[::3]
    [1, 4, 7]

In [ ]:
# Solution
%load solutions/Array-03.py